# calculate wind shear excedence map

In [ ]:
import iris

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

In [ ]:
direction = iris.load_cube('wind_direction.nc')
# direction

In [ ]:
speed = iris.load_cube('wind_speed.nc')
# speed

In [ ]:
from opscentretools import sheartools
shear = sheartools.calculate_shear(speed, direction)
shear

In [ ]:
shear_exceedance = shear.collapsed('realization', iris.analysis.PROPORTION,
                                    function=lambda values: values > 15)

In [ ]:
shear_exceedance

In [ ]:
%matplotlib inline
import iris.quickplot as qplt
import matplotlib.pyplot as plt
qplt.pcolormesh(shear_exceedance[2], cmap='viridis')
plt.gca().coastlines('50m', color='red')
plt.gca().set_extent([20, -10, 70, 30])
plt.gcf().set_size_inches(8, 12)

In [ ]:
import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs
import cartopy.feature as cf
from holoviews.operation.datashader import regrid
import panel as pn
pn.extension()
hv.extension('bokeh')

In [ ]:
def interactive_image(cube):
    dataset = gv.Dataset(cube, [coord.name() for coord in cube.dim_coords])
    image = regrid(dataset.to(gv.Image, ['longitude', 'latitude'], dynamic=True))
    coastlines = gv.feature.coastline
    return image.opts(colorbar=True, cmap='viridis', projection=ccrs.PlateCarree(), responsive=True, min_height=600, aspect=2)

In [ ]:
interactive_image(shear_exceedance)

In [ ]:
main_map = pn.panel(interactive_image(shear_exceedance) * gv.feature.coastline.opts(line_color='red'))

In [ ]:
main_map.show(websocket_origin='18.130.43.15:9999', port=9999)